In [1]:
import numpy as np
import pandas as pd
import random
import torch
import os
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [2]:
from datasets import load_dataset

In [3]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("friedrichor/MSR-VTT", "train_7k")['train']

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


msrvtt_train_7k.json:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

c:\Users\atom0\OneDrive\Documents\College_folder\Năm 4\CV\Code\Video_Summarization\sumenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\atom0\.cache\huggingface\hub\datasets--friedrichor--MSR-VTT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generating train split:   0%|          | 0/7010 [00:00<?, ? examples/s]

In [4]:
print(ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 7010
})


In [5]:
print(ds.column_names)
print(ds.features)

['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id']
{'video_id': Value('string'), 'video': Value('string'), 'caption': List(Value('string')), 'source': Value('string'), 'category': Value('int64'), 'url': Value('string'), 'start time': Value('float64'), 'end time': Value('float64'), 'id': Value('int64')}


In [ ]:
sampled_ds = ds.shuffle(seed=seed).select(range(200))
print(sampled_ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 7010
})


In [7]:
sample = sampled_ds[0]
print(sample["video_id"])
print(sample["caption"][:2])   # first two captions
print(sample["url"])
print(sample["start time"], sample["end time"])


video0
['a car is shown', 'a group is dancing']
https://www.youtube.com/watch?v=9lZi22qLlEo
137.72 149.44


In [8]:
data_path = "data/MSR-VTT"

In [9]:
os.makedirs(data_path, exist_ok=True)

In [10]:
import subprocess
failed_count = 0
def download_clip(clip_info):
    # Download and trim using ffmpeg
    url =  clip_info['url']
    start = clip_info['start time']
    end = clip_info['end time']
    out = os.path.join(data_path, f"{clip_info['video_id']}.mp4")
    global failed_count
    failed_count += 1
    if os.path.exists(out):
        print(f"File {out} already exists, skipping download.")
        return out
    try:
        cmd = [
            "yt-dlp", "-f", "bestvideo*+bestaudio/best",
            "--download-sections", f"*{start}-{end}",
            "-o", out,
            url
        ]
        subprocess.run(cmd, check=True)
        return out
    except Exception as e:
        print(f"Failed {clip_info['video_id']}: {e}")
        return ""
sampled_ds = sampled_ds.map(
    lambda x: {"clip_path": download_clip(x)},
)

Map:   0%|          | 0/7010 [00:00<?, ? examples/s]

Failed video4: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*268.58-278.83', '-o', 'data/MSR-VTT\\video4.mp4', 'https://www.youtube.com/watch?v=2q-dONPhzis']' returned non-zero exit status 1.
Failed video7: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*81.06-92.61', '-o', 'data/MSR-VTT\\video7.mp4', 'https://www.youtube.com/watch?v=y6sBoW139Sc']' returned non-zero exit status 1.
Failed video13: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*682.77-694.49', '-o', 'data/MSR-VTT\\video13.mp4', 'https://www.youtube.com/watch?v=bCUUP_Tm_w8']' returned non-zero exit status 1.
Failed video17: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*63.69-76.57', '-o', 'data/MSR-VTT\\video17.mp4', 'https://www.youtube.com/watch?v=wi3PGpmtxH8']' returned non-zero exit status 1.
Failed video23: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*

KeyboardInterrupt: 

In [13]:
missing_ds = sampled_ds.filter(lambda x: not x.get("clip_path"))
print(missing_ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 0
})


In [12]:
sampled_ds = sampled_ds.filter(
    lambda x: x.get("clip_path") not in ("", None)
)
print("Rows after drop:", len(sampled_ds))

Filter:   0%|          | 0/7010 [00:00<?, ? examples/s]

Rows after drop: 0


In [ ]:
sampled_ds.to_csv("data/sampled_msrvtt_clips.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

71419